### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [8]:
total_schools = len(school_data_complete["school_name"].unique())
total_students = len(school_data_complete["Student ID"])

total_budget = float(school_data["budget"].sum())

avg_math_score = round(float(school_data_complete["math_score"].mean()), 2)
avg_reading_score = round(float(school_data_complete["reading_score"].mean()), 2)

num_math_passing = len(school_data_complete.loc[school_data_complete["math_score"] >= 70]["Student ID"])
percent_math_passing = float(round(num_math_passing / total_students * 100, 2))
num_reading_passing = len(school_data_complete.loc[school_data_complete["reading_score"] >= 70]["Student ID"])
percent_reading_passing = float(round(num_reading_passing / total_students * 100, 2))
overall_passing_rate = round((percent_reading_passing + percent_math_passing) / 2, 2)

# create new dataframe
district_summary = pd.DataFrame({"Total Schools":total_schools, "Total Students":total_students,
                                "Total Budget":total_budget, "Average Math Score":avg_math_score,
                                "Average Reading Score":avg_reading_score,"Percent Passed Math":percent_math_passing,
                                "Percent Passed Reading":percent_reading_passing, "Overall Passing Rate":overall_passing_rate}, index=[0])

# clean up formatting
district_summary["Total Students"] = district_summary["Total Students"].map('{:,}'.format)
district_summary["Total Budget"] = district_summary["Total Budget"].map('${:,}'.format)
district_summary["Percent Passed Math"] = district_summary["Percent Passed Math"].map('{}%'.format)
district_summary["Percent Passed Reading"] = district_summary["Percent Passed Reading"].map('{}%'.format)
district_summary["Overall Passing Rate"] = district_summary["Overall Passing Rate"].map('{}%'.format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passed Math,Percent Passed Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.0",78.99,81.88,74.98%,85.81%,80.4%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [9]:
grouped_schools = school_data_complete.groupby("school_name")

# add data that's easier to get from school_data
school_summary = pd.DataFrame({"School Name":school_data["school_name"], "School Type":school_data["type"],
                             "Total Budget":school_data["budget"]})
school_summary = school_summary.set_index("School Name")
school_summary = school_summary.sort_values("School Name")

# do calculations on data from groupby object
school_summary["Total Students"] = grouped_schools["Student ID"].count()
school_summary["Budget Per Student"] = school_summary["Total Budget"] / school_summary["Total Students"]
school_summary["Average Math Score"] = round(grouped_schools["math_score"].mean(), 2)
school_summary["Average Reading Score"] = round(grouped_schools["reading_score"].mean(), 2)

# calculate percentages of students passing using apply
school_summary["Percent Passing Math"] = round((grouped_schools.apply(lambda x: (x["math_score"] >= 70).sum()) / 
                                          school_summary["Total Students"]) * 100, 2)
school_summary["Percent Passing Reading"] = round((grouped_schools.apply(lambda x: (x["reading_score"] >= 70).sum()) / 
                                          school_summary["Total Students"]) * 100, 2)
school_summary["Overall Passing Rate"] = round((school_summary["Percent Passing Math"] + 
                                               school_summary["Percent Passing Reading"]) / 2, 2)
school_summary = school_summary.sort_values("Overall Passing Rate", ascending=False)

# add formatting
school_summary["Total Budget"] = school_summary["Total Budget"].map('${:,}'.format)
school_summary.head()


,School Type,Total Budget,Total Students,Budget Per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"$1,081,356",1858,582.0,83.06,83.98,94.13,97.04,95.58
Thomas High School,Charter,"$1,043,130",1635,638.0,83.42,83.85,93.27,97.31,95.29
Pena High School,Charter,"$585,858",962,609.0,83.84,84.04,94.59,95.95,95.27
Griffin High School,Charter,"$917,500",1468,625.0,83.35,83.82,93.39,97.14,95.26
Wilson High School,Charter,"$1,319,574",2283,578.0,83.27,83.99,93.87,96.54,95.21


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [10]:
school_summary = school_summary.sort_values("Overall Passing Rate", ascending=True)

school_summary.head()

,School Type,Total Budget,Total Students,Budget Per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"$2,547,363",3999,637.0,76.84,80.74,66.37,80.22,73.30
Figueroa High School,District,"$1,884,411",2949,639.0,76.71,81.16,65.99,80.74,73.36
Huang High School,District,"$1,910,635",2917,655.0,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,"$3,094,650",4761,650.0,77.07,80.97,66.06,81.22,73.64
Ford High School,District,"$1,763,916",2739,644.0,77.10,80.75,68.31,79.30,73.81


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


In [11]:
ninth = school_data_complete.loc[school_data_complete["grade"] == "9th"]
tenth = school_data_complete.loc[school_data_complete["grade"] == "10th"]
eleventh = school_data_complete.loc[school_data_complete["grade"] == "11th"]
twelfth = school_data_complete.loc[school_data_complete["grade"] == "12th"]

math_ninth = ninth.groupby("school_name")["math_score"].mean()
math_tenth = tenth.groupby("school_name")["math_score"].mean()
math_eleventh = eleventh.groupby("school_name")["math_score"].mean()
math_twelfth = twelfth.groupby("school_name")["math_score"].mean()

math_grades = pd.DataFrame({"9th":math_ninth, "10th":math_tenth, 
                             "11th":math_eleventh, "12th":math_twelfth})

math_grades = round(math_grades, 4)

math_grades

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.0837,76.9968,77.5156,76.4922
Cabrera High School,83.0947,83.1545,82.7656,83.2775
Figueroa High School,76.4030,76.5400,76.8843,77.1514
Ford High School,77.3613,77.6723,76.9181,76.1800
Griffin High School,82.0440,84.2291,83.8421,83.3562
Hernandez High School,77.4385,77.3374,77.1360,77.1866
Holden High School,83.7874,83.4298,85.0000,82.8554
Huang High School,77.0273,75.9087,76.4466,77.2256
Johnson High School,77.1879,76.6911,77.4917,76.8632


## Reading Score by Grade 

* Perform the same operations as above for reading scores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


In [12]:
reading_ninth = ninth.groupby("school_name")["reading_score"].mean()
reading_tenth = tenth.groupby("school_name")["reading_score"].mean()
reading_eleventh = eleventh.groupby("school_name")["reading_score"].mean()
reading_twelfth = twelfth.groupby("school_name")["reading_score"].mean()

reading_grades = pd.DataFrame({"9th":reading_ninth, "10th":reading_tenth, 
                             "11th":reading_eleventh, "12th":reading_twelfth})

reading_grades = round(reading_grades, 4)

reading_grades

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.3032,80.9072,80.9456,80.9125
Cabrera High School,83.6761,84.2532,83.7884,84.2880
Figueroa High School,81.1986,81.4089,80.6403,81.3849
Ford High School,80.6327,81.2627,80.4036,80.6623
Griffin High School,83.3692,83.7069,84.2881,84.0137
Hernandez High School,80.8669,80.6601,81.3961,80.8571
Holden High School,83.6772,83.3246,83.8155,84.6988
Huang High School,81.2903,81.5124,81.4175,80.3060
Johnson High School,81.2607,80.7734,80.6160,81.2276


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


In [16]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-629", "$630-644", "$645-675"]

In [17]:
school_summary["Spending Ranges"] = pd.cut(school_summary["Budget Per Student"], spending_bins, labels=group_names)

spending_groups = school_summary.loc[:, ["Spending Ranges","Average Math Score", "Average Reading Score","Percent Passing Math", "Percent Passing Reading",
                                     "Overall Passing Rate"]].groupby("Spending Ranges")
spending_groups.mean()

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
Spending Ranges,,,,,
<$585,83.452500,83.935000,93.460000,96.610000,95.035000
$585-629,83.600000,83.885000,94.230000,95.900000,95.065000
$630-644,79.078333,81.891667,75.668333,86.106667,80.888333
$645-675,76.996667,81.026667,66.163333,81.133333,73.650000


## Scores by School Size

* Perform the same operations as above, based on school size.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [18]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 3000, 5000]
group_names = ["Small (<1000)", "Medium (1000-3000)", "Large (3000-5000)"]

In [19]:
school_summary["Size Ranges"] = pd.cut(school_summary["Total Students"], size_bins, labels=group_names)

size_groups = school_summary.loc[:, ["Size Ranges","Average Math Score", "Average Reading Score","Percent Passing Math", "Percent Passing Reading",
                                "Overall Passing Rate"]].groupby("Size Ranges")

size_groups.mean()

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
Size Ranges,,,,,
Small (<1000),83.820000,83.925000,93.550000,96.100000,94.825000
Medium (1000-3000),81.175556,82.935556,84.648889,91.316667,87.982222
Large (3000-5000),77.062500,80.917500,66.465000,81.057500,73.765000


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


In [20]:
district_groups = school_summary.loc[:, ["School Type", "Average Math Score", "Average Reading Score","Percent Passing Math", "Percent Passing Reading",
                                     "Overall Passing Rate"]].groupby("School Type")
district_groups.mean()

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.472500,83.897500,93.620000,96.586250,95.102500
District,76.955714,80.965714,66.548571,80.798571,73.675714
